In [1]:
# pip install pandas numpy matplotlib statsmodels scikit-learn
import pandas as pd, numpy as np, matplotlib.pyplot as plt, math
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

df = pd.read_csv("dados_onibus.csv")

# cria ordem de leitura
df["row_id"] = np.arange(len(df)) #para preservar a ordem original, e saber diferenciar a 1 segunda da 2 segunda
ordem_semanal = ["Seg","Ter","Qua","Qui","Sex","Sab","Dom"]
idx_semanal = {d:i for i,d in enumerate(ordem_semanal)}
df["idx_semanal"] = df["dia"].map(idx_semanal)

# ordena por linha e ordem de aparecimento
df = df.sort_values(["linha","row_id"]).reset_index(drop=True)

# contador sequencial de dias para cada linha, o eixo temporal do modelo.
df["t_seq"] = df.groupby("linha").cumcount()

capacidade_onibus = 50
resultados, previsoes = {}, []

for linha, g in df.groupby("linha"):
    # Para cada linha de ônibus, cria uma série temporal s indexada por t_seq
    s = g.set_index("t_seq")["requests"]

    # SARIMAX semanal
    # usa 1 valor passado, 1 diferenciacao (quantas vezes subtrair valores consecutivos para estabilizar a série) e 2 erro passado (para corrigir resultados)
    #usa também 1 valor de 7 dias atrás, com diferenciação sazonal
    model = SARIMAX(s, order=(1,1,2), seasonal_order=(1,1,0,7))
    res = model.fit(disp=False)

    # previsão 1 passo à frente
    fc = res.get_forecast(steps=1)
    media = float(fc.predicted_mean.iloc[0]) #valor previsto
    lo, hi = fc.conf_int().iloc[0].tolist() #limites inferior e superior do intervalo de confiança

    # erro de treino
    fitted = res.fittedvalues.dropna() #valores modelo
    y_real = s.loc[fitted.index] #valores reais
    mse = mean_squared_error(y_real, fitted) #Compara com os valores reais da série e mede o erro quadrático médio (MSE)

    prev_round = int(round(media))
    onibus = math.ceil(max(prev_round,0) / capacidade_onibus)

    resultados[linha] = dict(prev=prev_round, onibus=onibus, mse=mse, lo=lo, hi=hi, last=s.iloc[-1])
    previsoes.append({"linha": linha, "next_day": int(s.index[-1])+1, "requests": prev_round})

# DataFrame final
df_prev = pd.DataFrame(previsoes).sort_values("linha")

# Resumo por linha
for linha, r in resultados.items():
    print(f"Linha {linha}: prev={r['prev']} ônibus={r['onibus']}  MSE={r['mse']:.2f}  "
          f"(Intervalo≈[{r['lo']:.1f},{r['hi']:.1f}])")


c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statesp

Linha 1: prev=1236 ônibus=25  MSE=695271.31  (Intervalo≈[1201.2,1270.1])
Linha 2: prev=2376 ônibus=48  MSE=837775.12  (Intervalo≈[2138.0,2614.6])
Linha 3: prev=1082 ônibus=22  MSE=147454.95  (Intervalo≈[1010.6,1152.7])
Linha 4: prev=835 ônibus=17  MSE=148155.98  (Intervalo≈[730.7,938.6])
Linha 5: prev=231 ônibus=5  MSE=155372.39  (Intervalo≈[97.2,365.1])


c:\Users\leona\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
